# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [2]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [ ]:
#!pip install aqua

In [3]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [4]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the slurm job

In [5]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
    job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50
#SBATCH --error=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/logs/dask-worker-%j.err
#SBATCH --output=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/output/dask-worker-%j.out

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.3:43125 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35281 instead
  warnings.warn(
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config f

### Loading the slurm job to the queue 

In [6]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [8]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4592298   compute dask-wor  b382267  R       5:43      1 l40212
           4592287   compute dask-wor  b382267  R       7:32      1 l40221


0

### Canceling the job if it is necessary. Set the job ID manually

In [7]:
Job_ID =   4586525
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [9]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod

### Loading the extra function, which can be used in any diagnostic

In [49]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update,  data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update,  data_size
        break

### Loading the tropical precipitation module

In [11]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [12]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [13]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [14]:
import aqua

In [15]:
from aqua import Reader
from aqua.reader import catalogue

In [16]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# IFS data

In [17]:
reader = Reader(model="IFS", exp="tco3999-ng5", source="ICMGG_atm2d",configdir=configdir)
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
IFS = reader.retrieve()

In [18]:
IFS

<xarray.Dataset>
Dimensions:    (time: 5401, value: 64144000, level: 5)
Coordinates:
    lat        (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * level      (level) int64 1 2 3 4 5
    lon        (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2020-01-20 ... 2020-09-01
Dimensions without coordinates: value
Data variables: (12/81)
    100si      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    100u       (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    100v       (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    10u        (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    10v        (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    2d         (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    ...         ...
    uvb        (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vike       (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vipie      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vipile     (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vithe      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    z          (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

### size of entire dataset

In [20]:
data_size(IFS)

1732208720000

### timestep of dataset is

In [21]:
time_interpreter(IFS)

'1H'

### Selecting the small part of dataset for tests

In [22]:
IFS_small = IFS['tprate'][10:11,:]

### size of selected chunk

In [23]:
data_size(IFS_small)

64144000

### Attribute update

In [24]:
IFS_small = xarray_attribute_update(IFS_small, IFS)

### Load selected chunk into a memory

In [26]:
IFS_small = IFS_small.compute()

In [27]:
IFS_small

<xarray.DataArray 'tprate' (time: 1, value: 64144000)>
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.31136915e-11, 6.62273831e-11, 6.62273831e-11]])
Coordinates:
    lat      (value) float64 89.98 89.98 89.98 89.98 ... -89.98 -89.98 -89.98
    lon      (value) float64 0.0 18.0 36.0 54.0 72.0 ... 288.0 306.0 324.0 342.0
  * time     (time) datetime64[ns] 2020-01-20T10:00:00
Dimensions without coordinates: value
Attributes: (12/27)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    src_units:                  m
    units_fixed:                1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

### Units of precepitation rate

In [28]:
IFS_small.attrs['units']

'm s**-1'

In [29]:
time_interpreter(IFS_small) 

'False. Load more timesteps then one'

### Name of coordinates

In [30]:
diag.coordinate_names(IFS_small)

('lat', 'lon')

### Tropical latitudes

In [31]:
IFS_small = diag.ds_per_lat_range(IFS_small)

In [32]:
IFS_small

<xarray.DataArray 'tprate' (time: 1, value: 13435440)>
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.32454766e-10, 6.62273831e-11, 0.00000000e+00]])
Coordinates:
    lat      (value) float64 9.978 9.978 9.978 9.978 ... -9.978 -9.978 -9.978
    lon      (value) float64 0.0 0.02527 0.05055 0.07582 ... 359.9 359.9 360.0
  * time     (time) datetime64[ns] 2020-01-20T10:00:00
Dimensions without coordinates: value
Attributes: (12/27)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    src_units:                  m
    units_fixed:                1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

### Fastest histogram

In [33]:
hist_fast_IFS  = diag.hist1d_fast(IFS_small,  preprocess = False)
hist_fast_IFS

<xarray.DataArray (bin: 15)>
array([1.3420836e+07, 9.7620000e+03, 2.7920000e+03, 1.1610000e+03,
       4.6200000e+02, 2.2100000e+02, 1.1300000e+02, 3.5000000e+01,
       2.3000000e+01, 1.4000000e+01, 1.2000000e+01, 1.0000000e+00,
       5.0000000e+00, 3.0000000e+00, 0.0000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-06 1.333e-05 ... 8e-05 8.667e-05 9.333e-05
Attributes: (12/27)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    src_units:                  m
    units_fixed:                1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

### Plot of obtained histogram 

In [35]:
diag.hist_plot(hist_fast_IFS, pdf = True, smooth = False, label='IFS', plot_title = "ICON, precipitation rate for one timestep")

## Plot of mean and median value of precipitation rate

In [36]:
diag.mean_and_median_plot(IFS_small, savelabel = 'IFS')

## Image

In [39]:
IFS_small

<xarray.DataArray 'tprate' (time: 1, value: 13435440)>
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.32454766e-10, 6.62273831e-11, 0.00000000e+00]])
Coordinates:
    lat      (value) float64 9.978 9.978 9.978 9.978 ... -9.978 -9.978 -9.978
    lon      (value) float64 0.0 0.02527 0.05055 0.07582 ... 359.9 359.9 360.0
  * time     (time) datetime64[ns] 2020-01-20T10:00:00
Dimensions without coordinates: value
Attributes: (12/27)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    src_units:                  m
    units_fixed:                1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

In [ ]:
image_creator(IFS_small, vmin =0, vmax = 10**(-7),  label='IFS')

In [41]:
import matplotlib.pyplot as plt

In [48]:
IFS_small[0,:].plot()
plt.savefig("./figures/IFS_trop.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

In [ ]:

scale_factor = 90/diag.trop_lat
fig = plt.figure(figsize=(8*scale_factor,6))
diag.ds_per_lat_range(IFS)['tprate'][0].plot()#vmin=0, vmax=100)


plt.savefig("./figures/IFS_trop.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')